In [1]:
import urllib.request
import collections
import math
import os
import random
import zipfile
import datetime as dt
import numpy as np
import tensorflow as tf
import pandas as pd
import nltk
import re
from string import punctuation
import matplotlib.pyplot as plt
from nltk.util import ngrams
from itertools import chain
from sklearn.externals.joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import euclidean_distances
from pyemd import emd
import seaborn as sns
import matplotlib.pylab as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#tf.enable_eager_execution()

C:\Users\warlo\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import json
with open('word_index_dictionary_for_custom_embeddings_alpha_6_AUG_ADAM_MPE.json', 'r') as fp:
    dictionary_ = json.load(fp)


with open('word_index_dictionary_for_Generic_Skipgram_alpha_0.01_28_June.json', 'r') as fp:
    dictionary_generic = json.load(fp)
    
with open('word_index_dictionary_for_Google_embeddings_25_MAY.json', 'r') as fp:
    dictionary_google = json.load(fp)

    
with open('word_index_dictionary_eqpt_taxo_competetive_with_Google_6_AUG_corrected.json', 'r') as fp:
    dictionary_competitive_ = json.load(fp)
    
with open('word_index_dictionary_eqpt_taxo_for_competetive_with_skipgram_6_AUG.json', 'r') as fp:
    dictionary_competitive_skip = json.load(fp)
    
    
embeddings_5 = np.load('Embeddings_generated_alpha_5_6_AUG_ADAM_MPE_LE_NSC.npy')
norm_=np.sqrt(np.sum(np.square(embeddings_5), axis=1))
embeddings_5_scaled=embeddings_5/norm_[:,None]


embeddings_2 = np.load('Embeddings_generated_alpha_2_6_AUG_ADAM_MPE_LE_NSC.npy')
norm_=np.sqrt(np.sum(np.square(embeddings_2), axis=1))
embeddings_2_scaled=embeddings_2/norm_[:,None]

embeddings_35 = np.load('Embeddings_generated_alpha_35_6_AUG_ADAM_MPE_LE_NSC.npy')
norm_=np.sqrt(np.sum(np.square(embeddings_35), axis=1))
embeddings_35_scaled=embeddings_35/norm_[:,None]

embeddings_65 = np.load('Embeddings_generated_alpha_65_6_AUG_ADAM_FM_MPE_NSC.npy')
norm_=np.sqrt(np.sum(np.square(embeddings_65), axis=1))
embeddings_65_scaled=embeddings_65/norm_[:,None]


embeddings_8 = np.load('Embeddings_generated_alpha_80_6_AUG_ADAM_MPE_LE_NSC.npy')
norm_=np.sqrt(np.sum(np.square(embeddings_8), axis=1))
embeddings_8_scaled=embeddings_8/norm_[:,None]


embeddings_100 = np.load('Embeddings_generated_alpha_1_6_AUG_ADAM_MPE_LE_NSC.npy')
norm_=np.sqrt(np.sum(np.square(embeddings_100), axis=1))
embeddings_100_scaled=embeddings_100/norm_[:,None]


embeddings_google= np.load('Embeddings_generated_Google_25_MAY.npy')
norm_google=np.sqrt(np.sum(np.square(embeddings_google), axis=1))
embeddings_google_scaled=embeddings_google/norm_google[:,None]


embeddings_generic= np.load('Embeddings_generated_Generic_skipgram_alpha_0.01_28_June.npy')
norm_generic=np.sqrt(np.sum(np.square(embeddings_generic), axis=1))
embeddings_generic_scaled=embeddings_generic/norm_generic[:,None]


embedding_competitive_ = np.load('Embeddings_eqpt_taxo_competetive_with_Google_6_AUG_corrected.npy')
norm_competitive=np.sqrt(np.sum(np.square(embedding_competitive_), axis=1))
embedding_competitive_scaled=embedding_competitive_/norm_competitive[:,None]

embedding_competitive_skipgram = np.load('Embeddings_generated_eqpt_taxo_competetive_with_skipgram_6_AUG.npy')
norm_competitive_skipgram=np.sqrt(np.sum(np.square(embedding_competitive_skipgram), axis=1))
embedding_competitive_skipgram_scaled=embedding_competitive_skipgram/norm_competitive_skipgram[:,None]

In [3]:
from sklearn.externals.joblib import Parallel, delayed
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import euclidean_distances
from pyemd import emd
def _wmd(i, row, X_train, embedding_array_):
    """Compute the WMD between training sample i and given test row.

    Assumes that `row` and train samples are sparse BOW vectors summing to 1.
"""
    union_idx = np.union1d(X_train[i].indices, row.indices)
    W_minimal = embedding_array_[union_idx]
    W_dist = euclidean_distances(W_minimal)
    bow_i = X_train[i, union_idx].A.ravel()
    bow_j = row[:, union_idx].A.ravel()

    bow_i = bow_i.astype(np.double)
    bow_j = bow_j.astype(np.double)

    bow_i /= bow_i.sum()
    bow_j /= bow_j.sum()

    D_ = W_dist.astype(np.double)
    D_ /= D_.max()  # just for comparison purposes
    return emd(bow_i, bow_j, D_)
    
def _wmd_row(row, X_train, embedding_array_):
    """Wrapper to compute the WMD of a row with all training samples.

    Assumes that `row` and train samples are sparse BOW vectors summing to 1.
    Useful for parallelization.
    """
    n_samples_train = X_train.shape[0]
    return [_wmd(i, row, X_train, embedding_array_) for i in range(n_samples_train)]

def _pairwise_wmd(n_jobs, verbose, embedding_array_, X_test, X_train=None):
    """Computes the word mover's distance between all train and test points.

    Parallelized over rows of X_test.

    Assumes that train and test samples are sparse BOW vectors summing to 1.

    Parameters
    ----------
    X_test: scipy.sparse matrix, shape: (n_test_samples, vocab_size)
        Test samples.

    X_train: scipy.sparse matrix, shape: (n_train_samples, vocab_size)
        Training samples. If `None`, uses the samples the estimator was fit with.

    Returns
    -------
    dist : array, shape: (n_test_samples, n_train_samples)
        Distances between all test samples and all train samples.

    """
    n_samples_test = X_test.shape[0]

    if X_train is None:
        X_train = self._fit_X

    dist = Parallel(n_jobs=n_jobs, verbose=verbose, max_nbytes= None)(delayed(_wmd_row)(test_sample, X_train, embedding_array_)for test_sample in X_test)
    #dist = [_wmd_row(test_sample, X_train)for test_sample in X_test]
    
    return np.array(dist)

In [4]:
import time
def generate_distance_matrix (dictionary_, embeddings_, df, col_name):
    t1 = time.time()
    master_lst = []
    for i in range (len(df)):
        txt = df.loc[i,col_name]
        if txt != 'nan':
            master_lst.append(txt)
    vect = CountVectorizer(stop_words="english").fit(master_lst)
    common = [word for word in vect.get_feature_names() if word in dictionary_]
    vect = CountVectorizer(vocabulary=common, dtype=np.double)
    X_train = vect.fit_transform(master_lst)
    #print(X_train)
    new_dictionary_common = {}
    for word in common:
        new_dictionary_common[word] = vect.get_feature_names().index(word)
    #print(common)
    array_initialize = np.zeros((len(common), 100), float)
    for i, word in enumerate(common):
        dict_index = dictionary_[word]
        array_initialize[i] = embeddings_[dict_index]
    print(len(array_initialize))
    distance_matrix_ = _pairwise_wmd(5, False, array_initialize, X_train, X_train)
    t2 = time.time()
    print(t2-t1)
    return distance_matrix_

In [ ]:
'''
These sentences are the ones which were previously selected we first did the analysis on all sentences wihtout POS filteration

'''

In [ ]:
reselected_POS = pd.read_csv('filename.csv')
reselected_POS

In [7]:
distance_matrix_emd_5_scaled = generate_distance_matrix(dictionary_, embeddings_5_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_2_scaled  = generate_distance_matrix(dictionary_, embeddings_2_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_35_scaled  = generate_distance_matrix(dictionary_, embeddings_35_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_65_scaled  = generate_distance_matrix(dictionary_, embeddings_65_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_8_scaled  = generate_distance_matrix(dictionary_, embeddings_8_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_google_scaled = generate_distance_matrix(dictionary_google, embeddings_google_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
# distance_matrix_emd_generic_low_epoch_scaled = generate_distance_matrix(dictionary_generic_medium_epoch, embeddings_generic_medium_epoch_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_100_scaled = generate_distance_matrix(dictionary_, embeddings_100_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
# distance_matrix_emd_generic_high_epoch_scaled = generate_distance_matrix(dictionary_generic_high_epoch, embeddings_generic_high_epoch_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_generic_scaled = generate_distance_matrix(dictionary_generic, embeddings_generic_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_competitve_sclaed = generate_distance_matrix(dictionary_competitive_, embedding_competitive_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
distance_matrix_emd_competitve_skipgram_sclaed = generate_distance_matrix(dictionary_competitive_skip, embedding_competitive_skipgram_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')

1019
37.8800950050354
1019
32.50047707557678
1019
28.85258460044861
1019
28.14354419708252
1019
27.24311923980713
903
18.09295892715454
1019
28.331488370895386
1019
25.834645986557007
903
20.920730113983154
1019
29.583213090896606


In [9]:
from sklearn.cluster import KMeans
kmeans_alpha_5_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_5_scaled)
kmeans_alpha_2_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_2_scaled)
kmeans_alpha_35_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_35_scaled)
kmeans_alpha_65_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_65_scaled)
kmeans_alpha_8_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_8_scaled)
kmeans_google_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_google_scaled)
kmeans_generic_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_generic_scaled)
kmeans_alpha_1_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_100_scaled)
kmeans_competitive_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_competitve_sclaed)
kmeans_competitive_skipgram_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_competitve_skipgram_sclaed)

labels_a_5_scaled =  kmeans_alpha_5_scaled.labels_
labels_a_2_scaled =  kmeans_alpha_2_scaled.labels_
labels_a_35_scaled =  kmeans_alpha_35_scaled.labels_
labels_a_65_scaled =  kmeans_alpha_65_scaled.labels_
labels_a_8_scaled =  kmeans_alpha_8_scaled.labels_
labels_google_scaled =  kmeans_google_scaled.labels_
labels_generic_scaled =  kmeans_generic_scaled.labels_
labels_a_1_scaled =  kmeans_alpha_1_scaled.labels_
labels_competitive_scaled =  kmeans_competitive_scaled.labels_
labels_competitive_skipgram_scaled =  kmeans_competitive_skipgram_scaled.labels_

true_label = [*reselected_POS['True Labels']]

In [11]:
from sklearn import metrics
metric_alpha_50_scaled = metrics.adjusted_rand_score(true_label, labels_a_5_scaled)
metric_alpha_20_scaled = metrics.adjusted_rand_score(true_label, labels_a_2_scaled)
metric_alpha_35_scaled = metrics.adjusted_rand_score(true_label, labels_a_35_scaled)
metric_alpha_65_scaled = metrics.adjusted_rand_score(true_label, labels_a_65_scaled)
metric_alpha_80_scaled = metrics.adjusted_rand_score(true_label, labels_a_8_scaled)
metric_alpha_100_scaled = metrics.adjusted_rand_score(true_label, labels_a_1_scaled)
metric_google_scaled = metrics.adjusted_rand_score(true_label, labels_google_scaled)
metric_generic_scaled = metrics.adjusted_rand_score(true_label, labels_generic_scaled)
metric_competitive_scaled = metrics.adjusted_rand_score(true_label, labels_competitive_scaled)
metric_competitive_skipgram_scaled = metrics.adjusted_rand_score(true_label, labels_competitive_skipgram_scaled)

print('The results for short sentences without filtering is')
print('alpha 0.2 scaled:', metric_alpha_20_scaled)
print('alpha 0.35 scaled:', metric_alpha_35_scaled)
print('alpha 0.5 scaled:', metric_alpha_50_scaled)
print('alpha 0.65 scaled:', metric_alpha_65_scaled)
print('alpha 0.8 scaled:', metric_alpha_80_scaled)
print('alpha 1 scaled:', metric_alpha_100_scaled)
print('google scaled:', metric_google_scaled)
print('generic scaled:', metric_generic_scaled)
print('competetive with google scaled:', metric_competitive_scaled)
print('competetive with skipgram scaled:', metric_competitive_skipgram_scaled)

The results for short sentences without filtering is
alpha 0.2 scaled: 0.19148147943357313
alpha 0.35 scaled: 0.32119306711002493
alpha 0.5 scaled: 0.3415525332250708
alpha 0.65 scaled: 0.4200239800704493
alpha 0.8 scaled: 0.42327706798470577
alpha 1 scaled: 0.09561475338432324
google scaled: 0.11574939181327788
generic scaled: 0.19179961417154057
competetive with google scaled: 0.13142540102550346
competetive with skipgram scaled: 0.13494415317984954


In [12]:
'''np.save('Distance_Mat_Expt_2_re_CWEM_2_6_AUG', distance_matrix_emd_2_scaled)
np.save('Distance_Mat_Expt_2_re_CWEM_35_6_AUG', distance_matrix_emd_35_scaled)
np.save('Distance_Mat_Expt_2_re_CWEM_5_6_AUG', distance_matrix_emd_5_scaled)
np.save('Distance_Mat_Expt_2_re_CWEM_65_6_AUG', distance_matrix_emd_65_scaled)
np.save('Distance_Mat_Expt_2_re_CWEM_8_6_AUG', distance_matrix_emd_8_scaled)
np.save('Distance_Mat_Expt_2_re_CWEM_1_6_AUG', distance_matrix_emd_100_scaled)
np.save('Distance_Mat_Expt_2_re_google_6_AUG', distance_matrix_emd_google_scaled)
np.save('Distance_Mat_Expt_2_re_generic_6_AUG', distance_matrix_emd_generic_scaled)
np.save('Distance_Mat_Expt_2_re_google_competitive_6_AUG', distance_matrix_emd_competitve_sclaed)
np.save('Distance_Mat_Expt_2_re_generic_competitive_6_AUG', distance_matrix_emd_competitve_skipgram_sclaed)'''

In [15]:
from sklearn import metrics
from sklearn.cluster import KMeans
import json
with open('word_dictionary_eqpt_taxo_competetive_skipgram_10_AUG_updated_neg_sam.json', 'r') as fp:
    dictionary_competitive_skip_cp = json.load(fp)

embedding_competitive_skipgram_cp = np.load('Embeddings_eqpt_taxo_competetive_skipgram_10_AUG_updated_neg_sam.npy')
norm_competitive_skipgram_cp=np.sqrt(np.sum(np.square(embedding_competitive_skipgram_cp), axis=1))
embedding_competitive_skipgram_cp_scaled=embedding_competitive_skipgram_cp/norm_competitive_skipgram_cp[:,None]  

reselected_POS = pd.read_csv('reselected_sentences_after_POS_tag_17_July.csv')
distance_matrix_emd_competitve_skipgram_cp_sclaed = generate_distance_matrix(dictionary_competitive_skip_cp, embedding_competitive_skipgram_cp_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
kmeans_competitive_skipgram_cp_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_competitve_skipgram_cp_sclaed)
labels_competitive_skipgram_cp_scaled =  kmeans_competitive_skipgram_cp_scaled.labels_
true_label = [*reselected_POS['True Labels']]
metric_competitive_skipgram_cp_scaled = metrics.adjusted_rand_score(true_label, labels_competitive_skipgram_cp_scaled)
print('competetive with skipgram scaled changed_param:', metric_competitive_skipgram_cp_scaled)

1019
37.70107698440552
competetive with skipgram scaled changed_param: 0.1697002141327623


In [17]:
from sklearn import metrics
from sklearn.cluster import KMeans
import json
with open('word_index_dictionary_eqpt_taxo_competetive_with_Google_10_AUG_updated_neg_sam.json', 'r') as fp:
    dictionary_competitive_skip_cp = json.load(fp)

embedding_competitive_skipgram_cp = np.load('Embeddings_eqpt_taxo_competetive_with_Google_10_AUG_updated_neg_sam.npy')
norm_competitive_skipgram_cp=np.sqrt(np.sum(np.square(embedding_competitive_skipgram_cp), axis=1))
embedding_competitive_skipgram_cp_scaled=embedding_competitive_skipgram_cp/norm_competitive_skipgram_cp[:,None]  

reselected_POS = pd.read_csv('reselected_sentences_after_POS_tag_17_July.csv')
distance_matrix_emd_competitve_skipgram_cp_sclaed = generate_distance_matrix(dictionary_competitive_skip_cp, embedding_competitive_skipgram_cp_scaled, reselected_POS,'Final_Clubed_POS_lement_taxo_abbr')
kmeans_competitive_skipgram_cp_scaled = KMeans(n_clusters=5, random_state=0).fit(distance_matrix_emd_competitve_skipgram_cp_sclaed)
labels_competitive_skipgram_cp_scaled =  kmeans_competitive_skipgram_cp_scaled.labels_
true_label = [*reselected_POS['True Labels']]
metric_competitive_skipgram_cp_scaled = metrics.adjusted_rand_score(true_label, labels_competitive_skipgram_cp_scaled)
print('competetive with google scaled:', metric_competitive_skipgram_cp_scaled)

903
20.44896101951599
competetive with google scaled: 0.12924820092603498


# Bootstraping Results after Reselection with POS tags

In [13]:
from sklearn.cluster import KMeans
from sklearn import metrics

JobInstruc_df_sample_1 = pd.read_csv('reselected_sentences_after_POS_tag_17_July.csv')
m_a_2_=[]
m_a_5_=[]
m_a_35_=[]
m_a_65_=[]
m_a_8_=[]
m_a_1_=[]
m_goo_=[]
m_gene_=[]
m_gen_h_ep=[]
m_gen_l_ep=[]
m_c_goo_=[]
m_c_gen_=[]

for i in range(0,99):
    df_sub_sample = JobInstruc_df_sample_1.groupby('True Labels').apply(lambda x: x.sample(10)).reset_index(drop=True)
#     df_sub_sample.reset_index(drop=True, inplace = True)
#     k=len(df_sub_sample['True_Labels'].unique())
    k=5
    distance_matrix_emd_5_scaled = generate_distance_matrix(dictionary_, embeddings_5_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_2_scaled  = generate_distance_matrix(dictionary_, embeddings_2_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_35_scaled  = generate_distance_matrix(dictionary_, embeddings_35_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_65_scaled  = generate_distance_matrix(dictionary_, embeddings_65_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_8_scaled  = generate_distance_matrix(dictionary_, embeddings_8_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_google_scaled = generate_distance_matrix(dictionary_google, embeddings_google_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    #distance_matrix_emd_generic_low_epoch_scaled = generate_distance_matrix(dictionary_generic_medium_epoch, embeddings_generic_medium_epoch_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_100_scaled = generate_distance_matrix(dictionary_, embeddings_100_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    #distance_matrix_emd_generic_high_epoch_scaled = generate_distance_matrix(dictionary_generic_high_epoch, embeddings_generic_high_epoch_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_generic_scaled = generate_distance_matrix(dictionary_generic, embeddings_generic_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_competitve_sclaed = generate_distance_matrix(dictionary_competitive_, embedding_competitive_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')
    distance_matrix_emd_competitve_skipgram_sclaed = generate_distance_matrix(dictionary_competitive_skip, embedding_competitive_skipgram_scaled, df_sub_sample,'Final_Clubed_POS_lement_taxo_abbr')


    kmeans_alpha_5_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_5_scaled)
    kmeans_alpha_2_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_2_scaled)
    kmeans_alpha_35_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_35_scaled)
    kmeans_alpha_65_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_65_scaled)
    kmeans_alpha_8_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_8_scaled)
    kmeans_google_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_google_scaled)
    kmeans_generic_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_generic_scaled)
    kmeans_alpha_1_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_100_scaled)
   # kmeans_generic_high_epoch_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_generic_high_epoch_scaled)
   # kmeans_generic_low_epoch_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_generic_low_epoch_scaled)
    kmeans_competitive_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_competitve_sclaed)
    kmeans_competitive_skipgram_scaled = KMeans(n_clusters=k, random_state=0).fit(distance_matrix_emd_competitve_skipgram_sclaed)

    labels_a_5_scaled =  kmeans_alpha_5_scaled.labels_
    labels_a_2_scaled =  kmeans_alpha_2_scaled.labels_
    labels_a_35_scaled =  kmeans_alpha_35_scaled.labels_
    labels_a_65_scaled =  kmeans_alpha_65_scaled.labels_
    labels_a_8_scaled =  kmeans_alpha_8_scaled.labels_
    labels_google_scaled =  kmeans_google_scaled.labels_
    labels_generic_scaled =  kmeans_generic_scaled.labels_
    labels_a_1_scaled =  kmeans_alpha_1_scaled.labels_
   # labels_generic_high_epoch_scaled =  kmeans_generic_high_epoch_scaled.labels_
   # labels_generic_low_epoch_scaled =  kmeans_generic_low_epoch_scaled.labels_
    labels_competitive_scaled =  kmeans_competitive_scaled.labels_
    labels_competitive_skipgram_scaled =  kmeans_competitive_skipgram_scaled.labels_
    true_label = [*df_sub_sample['True Labels']]
    
    metric_alpha_50_scaled = metrics.adjusted_rand_score(true_label, labels_a_5_scaled)
    metric_alpha_20_scaled = metrics.adjusted_rand_score(true_label, labels_a_2_scaled)
    metric_alpha_35_scaled = metrics.adjusted_rand_score(true_label, labels_a_35_scaled)
    metric_alpha_65_scaled = metrics.adjusted_rand_score(true_label, labels_a_65_scaled)
    metric_alpha_80_scaled = metrics.adjusted_rand_score(true_label, labels_a_8_scaled)
    metric_alpha_100_scaled = metrics.adjusted_rand_score(true_label, labels_a_1_scaled)
    metric_google_scaled = metrics.adjusted_rand_score(true_label, labels_google_scaled)
    metric_generic_scaled = metrics.adjusted_rand_score(true_label, labels_generic_scaled)
   # metric_generic_high_epoch_scaled = metrics.adjusted_rand_score(true_label, labels_generic_high_epoch_scaled)
   # metric_generic_low_epoch_scaled = metrics.adjusted_rand_score(true_label, labels_generic_low_epoch_scaled)
    metric_competitive_scaled = metrics.adjusted_rand_score(true_label, labels_competitive_scaled)
    metric_competitive_skipgram_scaled = metrics.adjusted_rand_score(true_label, labels_competitive_skipgram_scaled)
    
    m_a_2_.append(metric_alpha_20_scaled)
    m_a_5_.append(metric_alpha_50_scaled)
    m_a_35_.append(metric_alpha_35_scaled)
    m_a_65_.append(metric_alpha_65_scaled)
    m_a_8_.append(metric_alpha_80_scaled)
    m_a_1_.append(metric_alpha_100_scaled)
    m_goo_.append(metric_google_scaled)
    m_gene_.append(metric_generic_scaled)
#     m_gen_h_ep.append(metric_generic_high_epoch_scaled)
#     m_gen_l_ep.append(metric_generic_low_epoch_scaled)
    m_c_goo_.append(metric_competitive_scaled)
    m_c_gen_.append(metric_competitive_skipgram_scaled)

685
6.471998453140259
685
6.401926040649414
685
6.347944736480713
685
6.62300181388855
685
6.595978498458862
604
3.888997793197632
685
5.84399676322937
685
5.733159303665161
604
4.23100471496582
685
6.653278827667236
709
8.293882608413696
709
7.231607437133789
709
7.512997627258301
709
7.748999357223511
709
8.505723714828491
644
5.5370917320251465
709
7.884904146194458
709
6.928999900817871
644
4.763998031616211
709
7.076998710632324
757
10.120168924331665
757
9.824333190917969
757
11.274513483047485
757
10.587936639785767
757
10.155556440353394
674
6.063000917434692
757
10.284856081008911
757
8.374993562698364
674
5.811397314071655
757
10.42882752418518
692
7.04408597946167
692
6.627627372741699
692
6.759978771209717
692
7.393524408340454
692
7.412084102630615
624
4.56491231918335
692
7.2080299854278564
692
6.854997634887695
624
5.632560729980469
692
7.4227893352508545
762
8.598997831344604
762
8.567576169967651
762
9.126305341720581
762
9.793821096420288
762
9.169992208480835
684
5.6

6.5689966678619385
692
10.381997346878052
680
9.872995853424072
680
10.30199670791626
680
10.112997055053711
680
9.94899606704712
680
9.585999250411987
614
6.478997707366943
680
9.815996646881104
680
8.178997993469238
614
6.016997814178467
680
9.658064603805542
732
12.306996822357178
732
11.089783668518066
732
11.823996305465698
732
11.978998899459839
732
11.368995666503906
664
7.673997640609741
732
11.13001823425293
732
10.020015001296997
664
7.637995958328247
732
11.508996725082397
668
9.513996362686157
668
7.7599968910217285
668
9.43899655342102
668
8.446996212005615
668
7.575997591018677
611
6.205997705459595
668
8.860997676849365
668
7.314678907394409
611
6.2443482875823975
668
8.674488544464111
655
7.990439176559448
655
7.126436233520508
655
7.572793960571289
655
8.491665124893188
655
7.733004093170166
596
5.5014002323150635
655
7.990706205368042
655
6.841435194015503
596
5.694896221160889
655
6.930992841720581
644
7.111000299453735
644
7.623996734619141
644
8.130995988845825
644

10.274682998657227
659
7.731838226318359
728
11.572996616363525
605
7.621996879577637
605
7.076997518539429
605
7.086998224258423
605
7.740001678466797
605
6.997995138168335
544
4.690005779266357
605
6.405998945236206
605
6.066996812820435
544
4.715996503829956
605
6.771999359130859
664
10.449995517730713
664
9.13112187385559
664
8.97422480583191
664
10.173997163772583
664
8.515996932983398
599
5.5359978675842285
664
8.569554090499878
664
8.503997087478638
599
6.403998851776123
664
9.64099645614624
704
9.853996276855469
704
8.94899845123291
704
9.848997831344604
704
9.99199628829956
704
9.805999517440796
634
6.149994373321533
704
8.54700231552124
704
8.077991962432861
634
6.261997699737549
704
8.936997175216675
629
6.784996032714844
629
7.861997842788696
629
7.725997447967529
629
7.038000822067261
629
7.574294090270996
562
5.819871664047241
629
6.955997705459595
629
6.311359882354736
562
5.271071434020996
629
7.5810065269470215
637
8.734757900238037
637
8.52726435661316
637
8.802236795

In [14]:
print('The results for short sentences without filtering is')
print('alpha 0.2 scaled:', np.mean(m_a_2_))
print('alpha 0.35 scaled:', np.mean(m_a_35_))
print('alpha 0.5 scaled:', np.mean(m_a_5_))
print('alpha 0.65 scaled:', np.mean(m_a_65_))
print('alpha 0.8 scaled:', np.mean(m_a_8_))
print('alpha 1 scaled:', np.mean(m_a_1_))
print('google scaled:', np.mean(m_goo_))
print('generic scaled:', np.mean(m_gene_))
#print('generic high epoch scaled:', np.mean(m_gen_h_ep))
#print('generic low epoch scaled:', np.mean(m_gen_l_ep))
print('competetive with google scaled:', np.mean(m_c_goo_))
print('competetive with skipgram scaled:', np.mean(m_c_gen_))

The results for short sentences without filtering is
alpha 0.2 scaled: 0.2669959995701372
alpha 0.35 scaled: 0.3689864029375312
alpha 0.5 scaled: 0.3298818684903033
alpha 0.65 scaled: 0.3604678491902795
alpha 0.8 scaled: 0.35604833455094975
alpha 1 scaled: 0.17243197224121934
google scaled: 0.11883366901482743
generic scaled: 0.20529094502328146
competetive with google scaled: 0.11686425235841504
competetive with skipgram scaled: 0.17012939227769336
